<a href="https://colab.research.google.com/github/Azevedo1996/ordem_automatica_compra_venda_binance/blob/main/Compra_e_venda_de_cripto_automatica_Binance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Instalar API binance.
!pip install python-binance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 KB 24.7 MB/s eta 0:00:00


In [4]:
# importando bibliotecas
import pandas as pd
import binance as bn

In [ ]:
% run | Notebooks/chaves_api.ipnyb

In [ ]:
# Utilizando a biblioteca do python binance
cliente = bn.Client(api_key, api_secret)
# Onde é api_key colocar a chave da API binance
# Onde é api_secret colocar a senha da API binance

In [ ]:
# Testar a conexão
cliente.get_account

In [ ]:
# Capturando dados
# Utilizando Bitcoin por ser o mais conhecido
# cliente.get_historical_klines('BTCUSDT', '1d', '1 Dec, 2018', '19 Feb 2023')
# realizando a verificação em curto prazo.
cliente.get_historical_klines('BTCUSDT', '1d', '3m', '30m')

In [ ]:
# ajustando colunas
df = pd.DataFrame(cliente.get_historical_klines('BTCUSDT', '1d', '3m', '30m'))
df = df.iloc[:, :2]
df.columns = ['date_open', 'Open', 'High', 'Low', 'Close', 'Volume']
df = df.set_index('date_open')
df.index = pd.to_datetime(df.index, unit='ms')
df = df.astype(float)
df

In [ ]:
# vizualizando a flutuação em dollar
df.Open.plot()

In [ ]:
from ast import Break
# Compra ou venda
compra = False
retorno_acum = (df.Open.pct_chance() + 1).cumprod()- 1
if not compra:
  if retorno_acum[-1] < -0.002:
    ordem = cliente.create_order('BTCUSDT', side='BUY', type='MARKET', quantity = 0.000001)
    print ('compra')
    compra = True
  else:
    print('Sem ordens de compra')
  if compra:
    while True:
      df = pd.DataFrame(cliente.get_historical_klines('BTCUSDT', '1d', '3m', '30m'))
      df = df.iloc[:, :2]
      df.columns = ['date_open', 'Open']
      df = df.set_index('date_open')
      df.index = pd.to_datetime(df.index, unit='ms')
      df = df.astype(float)
      
      ordem_executada = df.loc[df.index > pd.to_datetime(ordem['TransactTime'], unit='ms')]
      if len(ordem_executada) > 0:
        retorno_ordem_executada = (df.Open.pct_chance() + 1).cumprod()- 1
        if retorno_ordem_executada[-1] > 0.00015 or retorno_ordem_executada < - 0.00015:
          ordem = cliente.create_order('BTCUSDT', side='SELL', type='MARKET', quantity = 0.000001)
          print('Venda')
          break
